In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import prince
from sklearn.preprocessing import StandardScaler
import shap

In [ ]:
student_data = pd.read_csv("Dataset.csv")
student_data

In [ ]:
student_data.isnull().sum()

In [ ]:
student_data.dtypes

In [ ]:
student_data.fillna({"Fedu" : student_data["Fedu"].median() , "higher" : "yes","famsize" : "GT3" , "traveltime" : student_data["traveltime"].median() , "freetime" : student_data["freetime"].median(), "absences" : student_data["absences"].median()},inplace = True)
#print(student_data["absences"].median())
#print(student_data[student_data["romantic"]=="yes"].count())
#student_data.head(30)
student_data.drop(["G2"],axis=1,inplace = True)
student_data.dropna(axis = 0,inplace = True)
student_data

In [ ]:
romantic_true = student_data[student_data["romantic"] == "yes"]

In [ ]:
sns.histplot(data = romantic_true,x = "Feature_2")

In [ ]:
romantic_false = student_data[student_data["romantic"] == "no"]
sns.histplot(data = romantic_false, x = "Feature_2")

In [ ]:
category_mappings = {}
for col in student_data.columns:
    if student_data[col].dtype == "object":
        student_data[col] = student_data[col].astype("category")
        student_data[col+"_code"] = student_data[col].astype("category").cat.codes
        category_mappings[col] = dict(enumerate(student_data[col].cat.categories))
corr_matrix = student_data.corr(numeric_only = True)

plt.style.use("seaborn-v0_8-notebook")
plt.figure(figsize=(20, 10)) 
sns.heatmap(corr_matrix,annot = True , cmap = "coolwarm",fmt=".2f");


In [ ]:
# Feature_3 might represent frequency of off-campus visits because high correlation with goout and Dalc etc.
# Feature_2 has +ve impact on grades if grades are above a certain threshold
# Feature_3 varies inversely to feature_2... so feature 2 could be attendance etc.
#"Feature_1 varies inversely with schoolsup and grades and inversely with freetime. It could represent academic stress.

In [ ]:
category_mappings

In [ ]:
student_data

In [ ]:
sns.barplot(data = student_data ,x="G3",y="Feature_2",color = "Red",errorbar = None);

In [ ]:
sns.lineplot(data = student_data ,x="freetime",y="Feature_2",color = "Lawngreen");

In [ ]:
sns.lineplot(data = student_data ,x="freetime",y="Feature_1",color = "Magenta");

In [ ]:
sns.boxplot(data = student_data ,x="schoolsup",y="Feature_1",color = "Yellow");
#print(student_data["schoolsup"].dtype)


In [ ]:
sns.lineplot(data = student_data ,x="G1",y="Feature_1",color = "violet");

In [ ]:
# How are failures and grades correlated?
sns.barplot(data = student_data[student_data["G3"]!=5] ,x="G3",y="failures",color = "blue");

In [ ]:
# Students of which school have better grades? This gives an insight into quality of education in both schools
sns.boxplot(data = student_data, x="school",y = "G3",color = "orange");


In [ ]:
# How is travel time related to location of resdience of students? This gives us an idea of whether the school is located in a rural area or urban area
#Conclusion is that both schools are located in an urban area
sns.violinplot(data = student_data[student_data["school_code"] == 0], x="address" , y = "traveltime",color = "red")
plt.show()
sns.violinplot(data = student_data[student_data["school_code"] == 1], x="address" , y = "traveltime",color = "#a1c9f4");

In [ ]:
#Is there a correlation between grades and the interest in pursuing higher studies?
sns.boxplot(data = student_data, x="higher",y = "G3",color = "purple");

In [ ]:
#Internet vs school
mosaic(student_data,["school","internet"]);


In [ ]:
#Mjob vs Medu
sns.lineplot(data = student_data, x="Mjob",y = "Medu",color = "pink");

In [ ]:
#Fjob vs Fedu
sns.lineplot(data = student_data, x="Fjob",y = "Fedu",color = "red");

In [ ]:

# MANUAL TRY TO PERFORM MULTIVARIABLE GRADIENT DESCENT BUT OBVIOUSLY ITS TAKING A HELL LOT OF TIME :)


def sigmoid(x):
    return 1/(1+np.exp(-x))

def costFunction(w,b,data):
    cost = 0
    for i in range(0,len(data)):
        x = []
        yi = data.iloc[i,len(data.columns)-1]
        for j in range (0,len(data.columns)-1):
            x.append(data.iloc[i,j])
        x = np.array(x)
        #print("x is " ,end = "")
        #print(x)
        temp = np.dot(w,x) + b
        fxi = sigmoid(temp)
        cost = cost + yi*(np.log(fxi)) + (1-yi)*(np.log(1-fxi))
    cost = cost*(-1)/len(data)
    
    return cost
def gradient(w,b,data):
    grad = np.zeros(len(data.columns)-1)
    gradb = 0
    for i in range(0,len(data)):
        x = []
        yi = data.iloc[i,len(data.columns)-1]
        for j in range (0,len(data.columns)-1):
            x.append(data.iloc[i,j])
        x = np.array(x)
        #print("x is " ,end = "")
        #print(x)
        temp = np.dot(w,x) + b
        fxi = sigmoid(temp)
        factor = fxi-yi
        gradb = gradb + factor
        grad = grad + factor*x
    grad = grad/len(data)
    gradb = gradb/len(data)
    ans = [grad,gradb]
    return ans

def gradientDescent(w_init,b_init,data):
    gradW,gradB = gradient(w_init,b_init,data)
    alpha = 0.5
    while(np.linalg.norm(gradW)>0.001 and abs(gradB) >0.001):
        gradW,gradB = gradient(w_init,b_init,data)
        w_init = w_init - alpha*gradW
        b_init = b_init - alpha*gradB
        temp = [w_init,b_init]
        #print(temp)
    ans = [w_init,b_init]

    return ans
        

In [ ]:

# I DONT KNOW WHAT MCA IS BUT IT SEEMS TO BE LIKE PCA FOR CATGORICAL FIELDS. TRYING IT OUT:-

mca = prince.MCA(n_components=2)
student_dataLR = mca.fit_transform(student_data.drop("romantic_code",axis=1))
#student_dataLR =  PCA(n_components=2).fit_transform(student_data,numeric_only = True)
student_dataLR.reset_index(inplace= True,drop = True)
student_data.reset_index(inplace= True,drop = True)
student_dataLR["romantic_code"] = student_data["romantic_code"]
student_dataLR
#print(student_dataLR.iloc[0,len(student_dataLR.columns)-1])
# w = np.ones(len(student_dataLR)-1)
# b = 0
#print(gradient(w,b,student_dataLR))
# w,b = gradientDescent(w,b,student_dataLR)
# print(w)
# print(b)
      
      
#student_dataLR

In [ ]:
x_df = student_dataLR.drop("romantic_code",axis =1)
y_df = student_dataLR["romantic_code"]
y_arr = y_df.to_numpy()
x_arr = x_df.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x_arr, y_arr, test_size=0.2)
modelLR = LogisticRegression()
modelDT = DecisionTreeClassifier()
modelLR.fit(X_train, y_train)
y_predLR = modelLR.predict(X_test)
print("Accuracy of Logistic Regression :", accuracy_score(y_test, y_predLR))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_predLR))

modelDT.fit(X_train, y_train)
y_predDT = modelDT.predict(X_test)
print("Accuracy of Decision Tree:", accuracy_score(y_test, y_predDT))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_predDT))

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy of KNN :", accuracy_score(y_test, y_pred))

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print("Accuracy of RandForest", rf.score(X_test, y_test))


In [ ]:
#Feature scaling (optional)
#SelectKbest (k=2)
# models used -- LR , DT, RF, XGB


# TO DO -- UNDERSTAND FEATURE SCALING ,
#sELECTkBEST ,

#GRIDSEARCH , 

#PCA implementation ---- later 
# HOW TO PLOT DECISION BOUNDARY


#---------------------------------------------------------------------
# mca = prince.MCA(n_components=2)
# student_dataLR = mca.fit_transform(student_data.drop("romantic_code",axis=1))
#student_dataLR =  PCA(n_components=2).fit_transform(student_data,numeric_only = True)

#student_data.dtypes


# scaler = StandardScaler()
# numeric_student_data = scaler.fit_transform(numeric_student_data)
#pca = PCA(n_components = 2)

X = student_data.drop(["romantic_code"],axis=1)
#X = student_data.drop(["romantic_code","sex_code","school_code","reason_code"],axis = 1)
y = student_data["romantic_code"]






numeric_X = X.select_dtypes(include = "number")
selector = SelectKBest( k=10) #k = 10 is working really great
numeric_Xarr = selector.fit_transform(numeric_X,y)


selected_features = numeric_X.columns[selector.get_support()]
print("Selected features:", selected_features.tolist())



scaler = StandardScaler()
numeric_Xarr = scaler.fit_transform(numeric_Xarr)

#numeric_X = numeric_X[["goout","Feature_3"]]
#print(numeric_X.dtypes)




X_train, X_test, y_train, y_test = train_test_split(numeric_Xarr, y, test_size=0.2,random_state =69) #randomstate = 69 works reall great





#print(X_train)
# pca = PCA(n_components = 2)
# X_train = pca.fit_transform(X_train)
# X_test = pca.fit_transform(X_test)
# X = pca.fit_transform(numeric_X)
#print(X_train)
modelLR = LogisticRegression()
modelDT = DecisionTreeClassifier()
modelKNN = KNeighborsClassifier(n_neighbors=7)
modelRF = RandomForestClassifier()
modelXGB = xgb.XGBClassifier()
modelBayes = GaussianNB()

# modelLR.fit(X_train, y_train)
# y_predLR = modelLR.predict(X_test)
# print("Accuracy of Logistic Regression :", accuracy_score(y_test, y_predLR))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_predLR))

paramRF = {
    "n_estimators" : [25,30,35],
    "criterion" : ["gini","entropy","log_loss"],
     "min_samples_split" : [0.05,0.06],
     "min_samples_leaf" : [5,10,15] ,
     "max_depth" : [50,80]
}

grid = GridSearchCV(estimator = modelRF , param_grid = paramRF,cv = 6)
grid.fit(numeric_Xarr,y)
# Best model and parameters
print("Best parameters:", grid.best_params_)
print("Best CV score:", 100*grid.best_score_)
#print(modelRF.get_params())
# modelDT.fit(X_train, y_train)
# y_predDT = modelDT.predict(X_test)
# print("Accuracy of Decision Tree:", accuracy_score(y_test, y_predDT))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_predDT))


# modelKNN.fit(X_train, y_train)
# y_pred = modelKNN.predict(X_test)
# print("Accuracy of KNN :", accuracy_score(y_test, y_pred))


# modelBayes.fit(X_train, y_train)
# print("Accuracy of RandForest", modelBayes.score(X_test, y_test))

# print(modelBayes.get_params())
# modelXGB.fit(X_train,y_train)
# print("Accuracy of XGB", modelXGB.score(X_test, y_test))


# svc = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)
# svc.fit(X_train, y_train)

# # Predict
# y_pred = svc.predict(X_test)

# # Evaluate
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:}")

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(numeric_Xarr, y, test_size=0.2,random_state = 69) #randomstate = 69 works reall great

modelLR = LogisticRegression()
modelDT = DecisionTreeClassifier()
modelKNN = KNeighborsClassifier(n_neighbors=7)
modelRF = RandomForestClassifier()
modelXGB = xgb.XGBClassifier()

paramRF = {
    "n_estimators" : [20,40],
    "criterion" : ["gini","entropy","log_loss"],
     "min_samples_split" : [0.01,0.05,0.08],
     "min_samples_leaf" : [2,5,10] ,
     "max_depth" : [50,80]
}

grid = GridSearchCV(estimator = modelRF , param_grid = paramRF,cv = 4)



grid.fit(X_train, y_train)
print("Accuracy of RandForest", grid.score(X_test, y_test))
